<a href="https://colab.research.google.com/github/shadowusr/ml-course-2022/blob/main/1-model-selection-and-evaluation/model-selection-and-estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model selection and estimation

## Theory
The task of choosing the best model and estimating model specs is one of the most important steps in solving almost any machine learning problem.

Let's consider the most common model evaluation metrics:
- **Accuracy**
  
  Accuracy tells you how many times the model was able to detect a specific category. It shows share of correct answers out of total answers count.
  
  $$Accuracy = \frac{(TP+TN)}{(TP+FP+FN+TN)}$$
- **Precision**
  
  Precision measures the model's accuracy in classifying a sample as positive.
  $$Precision = \frac{TP}{(TP+FP)}$$
- **Recall**

  Recall measure the model's ability to detect positive samples.
  $$Recall = \frac{TP}{(TP+FN)}$$
- **Specificity**

  Specificity is the proportion of true negatives that are correctly predicted by the model.
  $$Specificity = \frac{TN}{(TN+FP)}$$
- **F1-score**

  F1 is a harmonic mean of the precision and recall
  $$F1\ Score = \frac{2*(Recall * Precision)}{(Recall + Precision)}$$
- **AUC/ROC (Receiver Operatin Characteristics) curve**

  AUC - ROC curve is a performance measurement for the classification problems at various threshold settings. ROC is a probability curve and AUC represents the degree or measure of separability. It tells how much the model is capable of distinguishing between classes.

To calculate these metrics, various cross-validation approaches can be used. To name a few:
- Leave-one-out
- K-fold
- Shuffle split

The end goal of model selection and estimation process is to choose the best model out of multiple options.

## Custom implementation
Our approach to model selection and estimation is to implement a framework with the following features:
- Easy configuration via file, configuration of dataset, models list and cross validation method
- Sequentally evaluate every given model on the dataset, build confusion matrices and calculate key quality metrics
- Print a human-readable report as table in the end

Let's look at our main class that performs models evaluation:

In [ ]:
import numpy as np
from sklearn.model_selection import LeaveOneOut, KFold, ShuffleSplit
from timeit import default_timer as timer

class AutoML:
    @staticmethod
    def __get_metrics_by_class(confusion_matrix: np.array):
        # A good article on confusion matrix:
        # https://towardsdatascience.com/confusion-matrix-for-your-multi-class-machine-learning-model-ff9aa3bf7826
        metrics = []
        classes_count = confusion_matrix.shape[0]

        for i in range(classes_count):
            tp = confusion_matrix[i][i]
            tn = np.sum(
                np.delete(
                    np.delete(confusion_matrix, i, 0),
                    i, 1
                )
            )
            fp = np.sum(np.delete(confusion_matrix[:, i], i, 0))
            fn = np.sum(np.delete(confusion_matrix[i], i, 0))

            metrics.append({
                "accuracy": (tp + tn) / (tp + tn + fp + fn),
                "precision": tp / (tp + fp),
                "recall": tp / (tp + fn),
                "specificity": tn / (tn + fp),
                "f1_score": (2 * tp) / (2 * tp + fp + fn),
            })

        return metrics

    @staticmethod
    def __get_average_metrics(metrics_by_class):
        result = {}
        metrics = ["accuracy", "precision", "recall", "specificity", "f1_score"]

        for metric in metrics:
            result[metric] = sum(item[metric] for item in metrics_by_class) / len(metrics_by_class)

        return result

    @staticmethod
    def get_reports(config):
        reports = []
        X = config["dataset"]["samples"]
        y = config["dataset"]["labels"]
        y_count = len(set(y))

        for model in config["models"]:
            start_time = timer()
            print(f"Evaluating model {model}")

            labels_dict = {k: v for v, k in enumerate(set(y))}
            label_to_index = lambda labels: [labels_dict.get(label) for label in labels]

            confusion_matrix = np.zeros((y_count, y_count))

            split = None

            if config["validation"]["method"] == "loo":
                loo = LeaveOneOut()
                split = loo.split(X)
            elif config["validation"]["method"] == "k_fold":
                kf = KFold(n_splits=config["validation"]["n_splits"])  # 2
                split = kf.split(X)
            elif config["validation"]["method"] == "shuffle_split":
                rs = ShuffleSplit(
                    n_splits=config["validation"]["n_splits"],  # 5
                    test_size=config["validation"]["test_size"],  # 0.25
                    random_state=0
                )
                split = rs.split(X)
                pass

            iteration = 0
            for train_index, test_index in split:
                print(f"\riteration: {iteration}", end='')
                X_train, X_test = X.loc[train_index], X.loc[test_index]
                y_train, y_test = y.loc[train_index], y.loc[test_index]

                # Calling fit() multiple times on the same model is fine:
                # https://stackoverflow.com/questions/49841324/what-does-calling-fit-multiple-times-on-the-same-model-do
                model.fit(X_train, y_train)

                y_predicted = model.predict(X_test)
                confusion_matrix[label_to_index(y_test), label_to_index(y_predicted)] += 1

                iteration += 1

            print("\n")
            end_time = timer()

            report = AutoML.__get_average_metrics(AutoML.__get_metrics_by_class(confusion_matrix))
            report["time"] = end_time - start_time

            reports.append(report)

        return reports


Now, let's write a configuration file:

In [ ]:
import os
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

__dir = os.path.dirname(os.path.realpath(__file__))

__df = pd.read_csv(f'{__dir}/diabetes.csv', encoding='utf-8', header=None)
__df.columns = [
    "TimesPregnant",
    "PlasmaGlucoseConcentration",
    "BloodPressure",
    "SkinfoldThickness",
    "Insulin",
    "BodyMassIndex",
    "DiabetesPedigreeFunction",
    "Age",
    "HasDiabetes"
]

config = {
    "dataset": {
        "samples": __df.drop(["HasDiabetes"], axis=1),
        "labels": __df.HasDiabetes,
    },
    "validation": {
        "method": "loo"
    },
    "models": [
        LogisticRegression(C=1, max_iter=500),
        KNeighborsClassifier(3),
        RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    ]
}


And combine it all together in the main file:

In [ ]:
from config import config
from auto_ml import AutoML
from prettytable import PrettyTable


reports = AutoML.get_reports(config)

table = PrettyTable()

table.field_names = ["Model", "Accuracy", "Precision", "Recall", "Specificity", "F1 Score", "Time (s)"]
for index, report in enumerate(reports):
    table.add_row([
        str(config["models"][index]),
        report["accuracy"],
        report["precision"],
        report["recall"],
        report["specificity"],
        report["f1_score"],
        report["time"]
    ])

print(table)


Here's sample report:

In [ ]:
+----------------------------------------------------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+
|                                Model                                 |      Accuracy      |     Precision      |       Recall       |    Specificity     |      F1 Score      |      Time (s)     |
+----------------------------------------------------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+
|                LogisticRegression(C=1, max_iter=500)                 | 0.7760416666666666 | 0.7612391193036354 | 0.7284477611940299 | 0.7284477611940299 | 0.7387982377739637 |    51.202216625   |
|                 KNeighborsClassifier(n_neighbors=3)                  | 0.6940104166666666 | 0.6614952413714024 | 0.6576567164179105 | 0.6576567164179105 | 0.6593425053652423 | 8.434406150000001 |
| RandomForestClassifier(max_depth=5, max_features=1, n_estimators=10) |        0.75        | 0.7357949218017478 | 0.6876716417910448 | 0.6876716417910448 | 0.698268876611418  |    30.257538501   |
+----------------------------------------------------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+

## TPOT
The Tree-Based Pipeline Optimization Tool (TPOT) was one of the very first AutoML methods and open-source software packages developed for the data science community.

The goal of TPOT is to automate the building of ML pipelines by combining a flexible expression tree representation of pipelines with stochastic search algorithms such as genetic programming. TPOT makes use of the Python-based scikit-learn library as its ML menu.

<img src="http://automl.info/wp-content/uploads/2017/07/tpot-pipeline-example-768x361.png">

Let's try using TPOT for solving the iris flowers classification problem.



In [ ]:
from tpot import TPOTClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

iris = load_iris()
iris.data[0:5], iris.target

X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target,
                                                    train_size=0.75, test_size=0.25)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

tpot = TPOTClassifier(verbosity=2, max_time_mins=2)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))

Sample output:

In [ ]:
Optimization Progress: 100%|██████████| 200/200 [00:18<00:00,  9.33pipeline/s]
Generation 1 - Current best internal CV score: 0.9825757575757577
Optimization Progress: 100%|██████████| 300/300 [00:28<00:00, 12.80pipeline/s]
Generation 2 - Current best internal CV score: 0.9825757575757577
Optimization Progress: 100%|██████████| 400/400 [00:39<00:00, 11.16pipeline/s]
Generation 3 - Current best internal CV score: 0.990909090909091
Optimization Progress: 100%|██████████| 500/500 [00:45<00:00, 17.87pipeline/s]
Generation 4 - Current best internal CV score: 0.990909090909091
Optimization Progress: 100%|██████████| 600/600 [00:52<00:00, 16.63pipeline/s]
Generation 5 - Current best internal CV score: 0.990909090909091
Optimization Progress: 100%|██████████| 700/700 [00:58<00:00, 17.42pipeline/s]
Generation 6 - Current best internal CV score: 0.990909090909091
Optimization Progress: 100%|██████████| 800/800 [01:06<00:00, 14.45pipeline/s]
Generation 7 - Current best internal CV score: 0.990909090909091
Optimization Progress: 100%|██████████| 900/900 [01:11<00:00, 12.45pipeline/s]
Generation 8 - Current best internal CV score: 0.990909090909091
Optimization Progress: 100%|██████████| 1000/1000 [01:17<00:00,  8.87pipeline/s]
Generation 9 - Current best internal CV score: 0.990909090909091
Optimization Progress: 100%|██████████| 1100/1100 [01:23<00:00, 11.12pipeline/s]
Generation 10 - Current best internal CV score: 0.990909090909091
Optimization Progress: 100%|██████████| 1200/1200 [01:29<00:00,  9.37pipeline/s]
Generation 11 - Current best internal CV score: 0.990909090909091
Optimization Progress: 100%|██████████| 1300/1300 [01:34<00:00,  9.55pipeline/s]
Generation 12 - Current best internal CV score: 0.990909090909091
Optimization Progress: 100%|██████████| 1400/1400 [01:40<00:00, 16.09pipeline/s]
Generation 13 - Current best internal CV score: 0.990909090909091
Optimization Progress: 100%|██████████| 1500/1500 [01:46<00:00,  9.96pipeline/s]
Generation 14 - Current best internal CV score: 0.990909090909091
Optimization Progress: 100%|██████████| 1600/1600 [01:52<00:00, 10.02pipeline/s]
Generation 15 - Current best internal CV score: 0.990909090909091
Optimization Progress: 100%|██████████| 1700/1700 [01:59<00:00,  7.28pipeline/s]
Generation 16 - Current best internal CV score: 0.990909090909091
                                                                                
TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: DecisionTreeClassifier(RBFSampler(input_matrix, RBFSampler__gamma=0.85), DecisionTreeClassifier__criterion=entropy, DecisionTreeClassifier__max_depth=3, DecisionTreeClassifier__min_samples_leaf=4, DecisionTreeClassifier__min_samples_split=9)
0.973684210526

## auto-sklearn
auto-sklearn is an automated machine learning toolkit and a drop-in replacement for a scikit-learn estimator. auto-sklearn frees a machine learning user from algorithm selection and hyperparameter tuning. It leverages recent advantages in Bayesian optimization, meta-learning and ensemble construction.

Let's apply auto-sklearn to the breast cancer classification problem.

In [ ]:
from pprint import pprint

import sklearn.datasets
import sklearn.metrics

import autosklearn.classification

X, y = sklearn.datasets.load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = \
    sklearn.model_selection.train_test_split(X, y, random_state=1)

automl = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=120,
    per_run_time_limit=30,
    tmp_folder='/tmp/autosklearn_classification_example_tmp',
)
automl.fit(X_train, y_train, dataset_name='breast_cancer')

Now we can conveniently view models scoreboard:

In [ ]:
print(automl.leaderboard())

# Outputs:
          rank  ensemble_weight                 type      cost  duration
model_id
7            1             0.08          extra_trees  0.014184  1.386039
16           2             0.06    gradient_boosting  0.021277  0.908949
21           3             0.02          extra_trees  0.021277  1.240055
2            4             0.04        random_forest  0.028369  1.507876
3            5             0.06                  mlp  0.028369  0.819861
22           6             0.02    gradient_boosting  0.028369  0.976891
10           7             0.06        random_forest  0.028369  1.648257
11           8             0.04        random_forest  0.028369  1.878659
13           9             0.04    gradient_boosting  0.028369  1.238531
26          10             0.06          extra_trees  0.028369  2.249340
19          11             0.08          extra_trees  0.028369  2.809216
27          12             0.06          extra_trees  0.028369  7.942544
8           13             0.02        random_forest  0.035461  1.692722
17          14             0.02    gradient_boosting  0.035461  1.413625
25          15             0.02             adaboost  0.042553  1.828053
9           16             0.02          extra_trees  0.042553  1.613578
30          17             0.04             adaboost  0.049645  0.595418
34          18             0.04          extra_trees  0.049645  1.247910
23          19             0.02                  mlp  0.049645  1.978261
15          20             0.04                  mlp  0.049645  3.234449
33          21             0.06        decision_tree  0.056738  0.868145
31          22             0.02          gaussian_nb  0.056738  0.643746
24          23             0.02        random_forest  0.070922  1.500164
20          24             0.04   passive_aggressive  0.078014  0.634079
32          25             0.02  k_nearest_neighbors  0.092199  0.678369

To view the final machine learning algorithms ensemble generated by auto-sklearn, we can do the following:

In [ ]:
pprint(automl.show_models(), indent=4)

# Outputs:
{   2: {   'balancing': Balancing(random_state=1),
           'classifier': <autosklearn.pipeline.components.classification.ClassifierChoice object at 0x7fb0ef3fef10>,
           'cost': 0.028368794326241176,
           'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7fb0ef79d3d0>,
           'ensemble_weight': 0.04,
           'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x7fb0ef3fea60>,
           'model_id': 2,
           'rank': 4,
           'sklearn_classifier': RandomForestClassifier(max_features=5, n_estimators=512, n_jobs=1,
                       random_state=1, warm_start=True)},
    3: {   'balancing': Balancing(random_state=1),
           'classifier': <autosklearn.pipeline.components.classification.ClassifierChoice object at 0x7fb0edc49430>,
           'cost': 0.028368794326241176,
           'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7fb0ef2df100>,
           'ensemble_weight': 0.06,
           'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x7fb0edc491f0>,
           'model_id': 3,
           'rank': 5,
           'sklearn_classifier': MLPClassifier(activation='tanh', alpha=0.0001363185819149026, beta_1=0.999,
              beta_2=0.9, early_stopping=True,
              hidden_layer_sizes=(115, 115, 115),
              learning_rate_init=0.00018009776276177523, max_iter=32,
              n_iter_no_change=32, random_state=1, verbose=0, warm_start=True)},
    ...

Now, let's get the score of the final ensemble:

In [ ]:
predictions = automl.predict(X_test)
print("Accuracy score:", sklearn.metrics.accuracy_score(y_test, predictions))
# Outputs:
Accuracy score: 0.958041958041958